# Import Libraries

In [1]:
from swarmintelligence import SlimeMouldAlgorithm, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']

In [3]:
gray_misc_dataset

{'AirplaneF16.tiff': array([[ 65, 199, 193, ..., 166, 164, 148],
        [ 54, 196, 192, ..., 179, 155, 140],
        [ 54, 197, 194, ..., 190, 159, 141],
        ...,
        [ 67, 213, 213, ..., 184, 176, 151],
        [ 68, 214, 214, ..., 166, 184, 167],
        [ 89,  86,  88, ..., 112, 111, 107]], dtype=uint8),
 'Lena.png': array([[162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        ...,
        [ 43,  43,  50, ..., 104, 100,  98],
        [ 44,  44,  55, ..., 104, 105, 108],
        [ 44,  44,  55, ..., 104, 105, 108]], dtype=uint8),
 'Male.tiff': array([[130, 138, 140, ..., 115, 113, 111],
        [139, 134, 131, ..., 110, 117, 116],
        [141, 127, 131, ..., 109, 108, 107],
        ...,
        [102, 103, 103, ...,  76,  80,  89],
        [106, 102, 109, ...,  72,  69,  82],
        [ 98, 105, 116, ...,  85,  81,  89]], dtype=uint8),
 'Mandrill.tiff': array([[145,  56,  49, ..., 113, 153, 177],

# Load Optimal Parameter

In [4]:
optimal_params = pickle.load(open('results/hyperparameter_tuning/SMA/optimal_params.pkl', 'rb'))
optimal_params

{'pop_size': 25, 'max_iteration': 100, 'z': 0.3}

# Evaluation

In [5]:
thresholds = [2,3,4,5]
fitness_functions = ['kapur_entropy']
objs = ['max']
n_runs = 30

In [6]:
sma_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds':[],
    'Mean CPU_time (seconds)': [],
    'Mean Fitness':[],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions': [],
    'CPU_time (seconds)': [],
    'Fitness':[],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in gray_misc_dataset.items():
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                sma_evaluation_results['image_name'].append(image_name)
                sma_evaluation_results['thresholds'].append(threshold)
                sma_evaluation_results['fitness_function'].append(fitness_function)
                sma_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                fitness_values = []
                list_best_thresholds = []
                list_fitness = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # PSO optimization
                    sma = SlimeMouldAlgorithm(
                        k=threshold,
                        slimeSize=optimal_params['pop_size'],
                        maxIteration=optimal_params['max_iteration'],
                        z=optimal_params['z'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    # Evaluation Metrics
                    st = time.time()
                    slimeMoulds, best_thresholds = sma.fit_run(image_array)
                    et = time.time()
                    regions = Utilization().digitize(image_array, best_thresholds)
                    list_cpu_time.append(et-st)
                    list_best_thresholds.append(best_thresholds)
                    fitness_values.append(sma.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, regions))
                    list_rmse.append(rmse(image_array, regions))
                    list_psnr.append(psnr(image_array, regions))
                    list_ssim.append(ssim(image_array, regions)[0])
                    list_uqi.append(uqi(image_array, regions))
                    print(sma.get_params_training_()['best_fitness_tracking'][-1])
                
                mean_best_thresholds = np.round(np.array(list_best_thresholds).mean(axis=0)).astype('int')
                mean_regions=Utilization().digitize(image_array, mean_best_thresholds)
                sma_evaluation_results['Mean best_thresholds'].append(mean_best_thresholds)
                sma_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                sma_evaluation_results['Mean Fitness'].append(np.mean(np.array(fitness_values)))
                sma_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                sma_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                sma_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                sma_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                sma_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                sma_evaluation_results['Regions'].append(mean_regions)
                sma_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                sma_evaluation_results['Fitness'].append(fitness_values)
                sma_evaluation_results['MSE'].append(list_mse)
                sma_evaluation_results['RMSE'].append(list_rmse)
                sma_evaluation_results['PSNR'].append(list_psnr)
                sma_evaluation_results['SSIM'].append(list_ssim)
                sma_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean(np.array(fitness_values))}")

	 AirplaneF16.tiff | Threshold =  2
11.917070706487834
12.197880097601033
11.894846459477591
11.939224749597196
11.126883322128633
11.926849930339875
11.95752893816907
11.929492523916725
12.210416342060524
12.206588846876762
12.196254131036532
12.176306588169071
11.395468823631646
12.104452267513867
12.201995832548945
12.15326116817873
12.149823010688163
12.029387296507089
12.097426888117226
12.166043144668683
11.688098301868502
12.009991041106035
12.059373018018338
11.994646444532464
11.855630597305286
12.182992843569279
11.948692543660904
11.94928772182965
11.995554137356416
11.895266808985653
Mean Fitness = 11.985224484198255
	 AirplaneF16.tiff | Threshold =  3
15.235974212483436
14.906538018682149
15.371039069517135
14.37548085614182
15.302028836812415
15.284146752085523
14.266257491829816
14.9480132132444
14.846470293091809
14.667427723284803
14.200307856403803
14.41138281828682
15.353346756734554
14.56888747758562
14.88902371844672
15.020600536085471
14.736692499945661
14.8039037

19.790432509075448
19.73674914409285
Mean Fitness = 19.890106236927018
	 Mandrill.tiff | Threshold =  2
5.100394928633376
5.100394928633376
5.100394928633376
5.100394928633376
5.100394928633376
5.100394928633376
11.965869525882217
5.100394928633376
5.100394928633376
5.100394928633376
5.100394928633376
5.100394928633376
5.100394928633376
5.100394928633376
5.100394928633376
11.986392491584567
5.100394928633376
5.100394928633376
5.100394928633376
5.100394928633376
5.100394928633376
5.100394928633376
5.100394928633376
5.100394928633376
11.749655379936307
5.100394928633376
5.100394928633376
5.100394928633376
5.100394928633376
5.100394928633376
Mean Fitness = 5.780419349016807
	 Mandrill.tiff | Threshold =  3
15.086484461217397
14.556017793459542
14.304533787111682
12.393724645154805
13.07804430684802
14.853710937371158
14.958626239894848
14.302354787589712
15.128175419732859
15.128119884651738
14.671909209177777
13.678019180673584
15.126519904443136
14.655329492507885
14.613815925895777
14.

20.44508181897079
18.582103277194605
17.307249464738472
19.884725237406617
20.17934141089709
19.226151423556303
16.208504837701362
19.110337799129
20.779744454539117
Mean Fitness = 18.831114440324395


## Visualize Results using DataFrame

In [7]:
sma_evaluation_results_df =pd.DataFrame(sma_evaluation_results)
sma_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds,Mean CPU_time (seconds),Mean Fitness,Mean MSE,Mean RMSE,Mean PSNR,Mean SSIM,Mean UQI,Regions,CPU_time (seconds),Fitness,MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,kapur_entropy,max,"[85, 163]",2.219229,11.985224,2783.683524,52.462720,13.780108,0.746364,0.926446,"[[85, 255, 255, 255, 255, 255, 255, 255, 255, ...","[2.1121442317962646, 1.2280089855194092, 1.293...","[11.917070706487834, 12.197880097601033, 11.89...","[3101.588878631592, 2786.027317047119, 2864.71...","[55.691910351788, 52.78283165051984, 53.523036...","[13.21496130115852, 13.680949904940132, 13.559...","[0.6881960508534166, 0.7596842502903289, 0.758...","[0.912308692016722, 0.922298289706957, 0.92141..."
1,AirplaneF16.tiff,3,kapur_entropy,max,"[77, 132, 175]",2.194872,14.675533,2233.747962,47.054448,14.716046,0.763558,0.941457,"[[77, 255, 255, 255, 255, 255, 255, 255, 255, ...","[1.9263405799865723, 3.177093744277954, 1.9671...","[15.235974212483436, 14.906538018682149, 15.37...","[1802.0994873046875, 1946.2602272033691, 2103....","[42.4511423557092, 44.11643942118821, 45.86311...","[15.573015977571751, 15.238794531561815, 14.90...","[0.7718946391678301, 0.7924101695360483, 0.801...","[0.9543726160700623, 0.9485523839739836, 0.946..."
2,AirplaneF16.tiff,4,kapur_entropy,max,"[55, 107, 162, 194]",1.987245,16.842527,1581.725298,38.853869,16.554610,0.748625,0.955692,"[[107, 255, 194, 194, 194, 194, 194, 194, 194,...","[2.219001054763794, 2.3568317890167236, 2.3112...","[17.03360797517624, 16.77554410743321, 16.0114...","[1941.4060668945312, 2402.8644790649414, 1148....","[44.06138975219156, 49.019021604525534, 33.882...","[15.249639784073025, 14.32351083469744, 17.531...","[0.6637051088775556, 0.7380883806526379, 0.698...","[0.9408685388584268, 0.9340924385757114, 0.965..."
3,AirplaneF16.tiff,5,kapur_entropy,max,"[52, 93, 131, 172, 203]",1.683900,18.599112,1309.582174,35.239442,17.438928,0.752443,0.962032,"[[93, 203, 203, 203, 203, 203, 203, 203, 203, ...","[1.8044548034667969, 1.8599779605865479, 1.332...","[20.607027333974155, 19.15615699113277, 17.734...","[1386.2436485290527, 1015.7348442077637, 1817....","[37.23229308717169, 31.870595291079262, 42.637...","[16.712407915502638, 18.063000099344794, 15.53...","[0.7503637888682163, 0.738292101728336, 0.7656...","[0.9650375910597202, 0.9675855679467555, 0.956..."
4,Lena.png,2,kapur_entropy,max,"[98, 168]",1.413131,11.966151,3167.802614,55.368556,13.396488,0.596723,0.859073,"[[168, 168, 168, 168, 168, 168, 168, 168, 168,...","[1.356858491897583, 1.5059831142425537, 2.4568...","[11.094936476844751, 12.197472242214783, 12.07...","[4959.814716339111, 2376.2484970092773, 2333.6...","[70.4259520087525, 48.746779350119915, 48.3079...","[11.176149080016053, 14.3718850568806, 14.4504...","[0.5162645835266411, 0.6363838580792555, 0.592...","[0.8046484683806246, 0.8764933451090308, 0.860..."
5,Lena.png,3,kapur_entropy,max,"[96, 139, 189]",1.650337,14.529874,2051.946740,44.448865,15.316694,0.632477,0.890525,"[[189, 189, 189, 189, 189, 189, 189, 189, 189,...","[1.9852855205535889, 1.5172033309936523, 1.221...","[14.39707013462202, 13.559533377157118, 14.555...","[2310.199131011963, 4559.825798034668, 1868.05...","[48.06453090390005, 67.52648219798414, 43.2209...","[14.494309447106733, 11.541321095186968, 15.41...","[0.5969500628263049, 0.5574117996752704, 0.632...","[0.8764008797032934, 0.8317914021941123, 0.895..."
6,Lena.png,4,kapur_entropy,max,"[68, 110, 155, 198]",1.694218,16.901794,1643.532611,39.421894,16.442310,0.672488,0.917424,"[[198, 198, 198, 198, 198, 198, 198, 198, 198,...","[1.5311682224273682, 1.244875431060791, 1.2267...","[16.93050077755855, 16.171650392550898, 16.164...","[1175.833797454834, 2111.7125511169434, 2606.1...","[34.29043303101951, 45.95337366414944, 51.0500...","[17.427344217377712, 14.884455596765282, 13.97...","[0.6818304968984767, 0.6334376252912891, 0.630...","[0.9185985955059524, 0.9038803835248544, 0.897..."
7,Lena.png,5,kapur_e

# Save results

In [8]:
pickle.dump(sma_evaluation_results_df, open('results/evaluation/SMA/SMA_kapur_df.pkl', 'wb'))
pickle.dump(sma_evaluation_results, open('results/evaluation/SMA/SMA_kapur_dict.pkl', 'wb'))